<a href="https://colab.research.google.com/github/greywolf8/fcc_predict_health_costs_with_regression.ipynb/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()
# Convert categorical columns to numerical values
dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})
dataset['smoker'] = dataset['smoker'].map({'yes': 1, 'no': 0})
dataset['region'] = dataset['region'].astype('category').cat.codes

# Separate features and labels
features = dataset.drop(columns=['expenses'])
labels = dataset['expenses']

# Split the data into training and testing datasets (80% train, 20% test)
train_dataset, test_dataset, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Normalize the feature datasets
scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

In [ ]:
# Create the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(train_dataset.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer with 1 neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

# Train the model
history = model.fit(
    train_dataset, train_labels,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    verbose=1
)
# Evaluate the model
test_loss, test_mae = model.evaluate(test_dataset, test_labels, verbose=1)
print(f"Mean Absolute Error on Test Dataset: {test_mae}")

# Predict healthcare costs using the test dataset
predicted_costs = model.predict(test_dataset).flatten()

# Plot the results
plt.figure(figsize=(8, 8))
plt.scatter(test_labels, predicted_costs, alpha=0.5)
plt.plot([0, max(test_labels)], [0, max(test_labels)], color='red')  # Perfect prediction line
plt.xlabel('True Expenses')
plt.ylabel('Predicted Expenses')
plt.title('True vs Predicted Healthcare Costs')
plt.show()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if test_mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
